<a href="https://colab.research.google.com/github/sinajahangir/HydrologicalDL_Colab/blob/main/KAN_CONUS_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!pip install tensorflow-decision-forests
import tensorflow_decision_forests as tfdf
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 118.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 125.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Succe

In [ ]:
# prompt: mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read data

In [ ]:
csv_path_train='/content/drive/MyDrive/PHIMP/All421_TrainTest/All421data_Train_v1.csv'
csv_path_test='/content/drive/MyDrive/PHIMP/All421_TrainTest/All421data_Test_v1.csv'
df_train = pd.read_csv(csv_path_train)
df_test = pd.read_csv(csv_path_test)

In [ ]:
df_train.tail()

,date,pr,srad,tmax,tmin,vp,q,basin_id,average_pr,average_q,average_tmax,average_tmin
3608807,2007-11-20,6.6,152.75,7.10,-1.82,536.88,27.62,420,5.450285,7.414476,14.486049,4.866612
3608808,2007-11-21,0.0,232.52,11.45,-0.07,608.75,17.68,420,5.450285,7.414476,14.486049,4.866612
3608809,2007-11-22,0.0,228.97,11.89,0.45,629.82,12.73,420,5.450285,7.414476,14.486049,4.866612
3608810,2007-11-23,0.0,244.66,15.05,0.48,629.72,9.90,420,5.450285,7.414476,14.486049,4.866612
3608811,2007-11-24,0.0,191.25,11.66,2.91,752.85,8.09,420,5.450285,7.414476,14.486049,4.866612


In [ ]:
df_test.tail()

,date,pr,srad,tmax,tmin,vp,q,basin_id,average_pr,average_q,average_tmax,average_tmin
636968,2012-01-11,0.00,188.99,11.73,3.85,804.71,4.53,420,5.450285,7.414476,14.486049,4.866612
636969,2012-01-12,0.00,225.65,13.29,2.11,714.24,4.32,420,5.450285,7.414476,14.486049,4.866612
636970,2012-01-13,0.00,233.53,14.33,2.02,710.75,4.11,420,5.450285,7.414476,14.486049,4.866612
636971,2012-01-14,0.00,207.19,10.63,1.87,698.56,3.94,420,5.450285,7.414476,14.486049,4.866612
636972,2012-01-15,9.07,106.43,3.53,-1.60,546.38,3.84,420,5.450285,7.414476,14.486049,4.866612


In [ ]:
def split_sequence_multi_train(sequence_x,sequence_y, n_steps_in, n_steps_out,mode='seq'):
    """
    written by:SJ
    sequence_x=features; 2D array
    sequence_y=target; 2D array
    n_steps_in=IL(lookbak period);int
    n_steps_out=forecast horizon;int
    mode:either single (many to one) or seq (many to many).
    This function creates an output in shape of (sample,IL,feature) for x and
    (sample,n_steps_out) for y
    """
    X, y = list(), list()
    k=0
    sequence_x=np.copy(np.asarray(sequence_x))
    sequence_y=np.copy(np.asarray(sequence_y))
    for _ in range(len(sequence_x)):
		# find the end of this pattern
        end_ix = k + n_steps_in
        out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
        if out_end_ix > len(sequence_x):
            break
		# gather input and output parts of the pattern
        seq_x = sequence_x[k:end_ix]
        #mode single is used for one output
        if n_steps_out==0:
            seq_y= sequence_y[end_ix-1:out_end_ix]
        elif mode=='single':
            seq_y= sequence_y[out_end_ix-1]
        else:
            seq_y= sequence_y[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y.flatten())
        k=k+1

    XX,YY= np.asarray(X), np.asarray(y)
    if (n_steps_out==0 or n_steps_out==1):
        YY=YY.reshape((len(XX),1))
    return XX,YY

In [ ]:
mean_=np.asarray(df_train.iloc[:,1:].mean())
std_=np.asarray(df_train.iloc[:,1:].std())

In [ ]:
df_test_tr=df_test.iloc[:,1:]-mean_
df_test_tr=df_test_tr/std_

In [ ]:
df_test_tr.head()

,pr,srad,tmax,tmin,vp,q,basin_id,average_pr,average_q,average_tmax,average_tmin
0,-0.414810,-1.314938,-1.571585,-1.384286,-1.027523,0.060093,-1.727941,0.164944,0.316118,-0.951796,-0.764845
1,-0.414810,-1.321855,-1.104175,-0.888873,-0.803962,0.019014,-1.727941,0.164944,0.316118,-0.951796,-0.764845
2,2.274691,-1.818817,-0.806566,-0.372484,-0.499100,0.312045,-1.727941,0.164944,0.316118,-0.951796,-0.764845
3,-0.414810,-1.216401,-0.954457,-0.890870,-0.818998,0.542088,-1.727941,0.164944,0.316118,-0.951796,-0.764845
4,-0.399464,-1.172727,-1.318709,-1.340338,-1.015989,0.459930,-1.727941,0.164944,0.316118,-0.951796,-0.764845


In [ ]:
df_test_tr=df_test_tr.drop(columns=['basin_id'])

In [ ]:
# prompt: add the basin_id column of df_test to df_test_tr
df_test_tr['basin_id'] = df_test['basin_id']

In [ ]:
mean_q=df_train['q'].mean()
std_q=df_train['q'].std()

In [ ]:
df_test_tr.head()

,pr,srad,tmax,tmin,vp,q,average_pr,average_q,average_tmax,average_tmin,basin_id
0,-0.414810,-1.314938,-1.571585,-1.384286,-1.027523,0.060093,0.164944,0.316118,-0.951796,-0.764845,0
1,-0.414810,-1.321855,-1.104175,-0.888873,-0.803962,0.019014,0.164944,0.316118,-0.951796,-0.764845,0
2,2.274691,-1.818817,-0.806566,-0.372484,-0.499100,0.312045,0.164944,0.316118,-0.951796,-0.764845,0
3,-0.414810,-1.216401,-0.954457,-0.890870,-0.818998,0.542088,0.164944,0.316118,-0.951796,-0.764845,0
4,-0.399464,-1.172727,-1.318709,-1.340338,-1.015989,0.459930,0.164944,0.316118,-0.951796,-0.764845,0


In [ ]:
inputs = tf.keras.layers.Input(shape=(365, 9))
#x=tf.keras.layers.Normalization(axis=-1)(inputs)
x = tf.keras.layers.LSTM(256, return_sequences=False)(inputs)
x=tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1,activation='linear')(x)
#outputs=tf.keras.layers.Reshape((3,1),name='output')(outputs)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), loss="mse")

# Load model

In [ ]:
def forecast_wrapper(array,n_steps_out=7):
    """
    Written by:SJ
    array:predictions;2D vector
    n_steps_out:number of steps out;int
    This function outputs a list with n_steps_out elements containing stepwise forecasts
    returns:list of forecasts
    """
    all_steps=[]
    for ii in range(0,n_steps_out):
        all_=array[ii:len(array):n_steps_out]
        all_steps.append(all_[n_steps_out-1-ii:len(all_)-ii])
    return all_steps

In [ ]:
# prompt: write nash-sutcliffe error index

def nash_sutcliffe_error(Q_obs,Q_sim):
    """
    Written by: SJ
    Q_obs: observed discharge; 1D vector
    Q_sim: simulated discharge; 1D vector
    This function calculates the NSE between observed and simulated discharges
    returns: NSE; float
    """
    if len(Q_sim)!=len(Q_obs):
        print('Length of simulated and observed discharges do not match')
        return
    else:
        num=np.sum(np.square(Q_sim-Q_obs))
        den=np.sum(np.square(Q_obs-np.mean(Q_obs)))
        NSE=1-(num/den)
        return NSE


In [ ]:
def i_normalize(x_tr,mean_,sd_):
  """
  Written by:SJ
  mean_,sd_,:mean_, and sd_ used for initial transformation; 2D array
  x_tr:transformed input
  This function inverses the transformation
  returns:inverse transfomed
  """
  x_i=x_tr*sd_+mean_
  return x_i

In [ ]:
def CC(Pr,Y):
    from scipy import stats
    Pr=np.reshape(Pr,(-1,1))
    Y=np.reshape(Y,(-1,1))
    return stats.pearsonr(Pr.flatten(),Y.flatten())[0]
def KGE(prediction,observation):

    nas = np.logical_or(np.isnan(prediction), np.isnan(observation))
    pred=np.copy(np.reshape(prediction,(-1,1)))
    obs=np.copy(np.reshape(observation,(-1,1)))
    r=CC(pred[~nas],obs[~nas])
    beta=np.nanmean(pred)/np.nanmean(obs)
    gamma=(np.nanstd(pred)/np.nanstd(obs))/beta
    kge=1-((r-1)**2+(beta-1)**2+(gamma-1)**2)**0.5
    return kge

In [ ]:
nse_scores_same_lstm=[]
kge_scores_same_lstm=[]

nse_scores_diff_lstm=[]
kge_scores_diff_lstm=[]


for jj in range(0,10):
  list_=list(pd.read_csv("/content/drive/MyDrive/PHIMP/LSTM_All_50_Weights_RandomNumbers/random_numbers_sim_%d.csv"%(jj)).iloc[:,1])
  model.load_weights("/content/drive/MyDrive/PHIMP/LSTM_All_50_Weights_RandomNumbers/Final_sim_%d.h5"%(jj))

  nse_scores_same_lstm.append([])
  kge_scores_same_lstm.append([])
  nse_scores_diff_lstm.append([])
  kge_scores_diff_lstm.append([])
  for ii in range(0,420):
    temp_x=np.asarray(df_test_tr[df_test_tr['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
    temp_y=np.asarray(df_test_tr[df_test_tr['basin_id']==ii]['q']).reshape((-1,1))
    xx,yy=split_sequence_multi_train(temp_x,temp_y,365,0,mode='seq')


    y_out=yy[:,0].reshape((-1,1))
    y_pred=model.predict(xx)
    y_m_out=y_pred.reshape((-1,1))

    y_out=i_normalize(y_out,mean_q,std_q)[:,0].reshape((-1,1))
    y_m_out=i_normalize(y_m_out,mean_q,std_q)

    if ii in list_:
      nse_scores_same_lstm[jj].append(nash_sutcliffe_error(y_out,y_m_out[:,0].reshape((-1,1))))
      kge_scores_same_lstm[jj].append(KGE(y_m_out[:,0].reshape((-1,1)),y_out))
    else:
      nse_scores_diff_lstm[jj].append(nash_sutcliffe_error(y_out,y_m_out[:,0].reshape((-1,1))))
      kge_scores_diff_lstm[jj].append(KGE(y_m_out[:,0].reshape((-1,1)),y_out))







In [ ]:
df_same_lstm = pd.DataFrame(nse_scores_same_lstm)
df_same_lstm.to_csv('/content/drive/MyDrive/PHIMP/LSTM_sim_same_nse.csv', index=False)

In [ ]:
df_same_lstm = pd.DataFrame(kge_scores_same_lstm)
df_same_lstm.to_csv('/content/drive/MyDrive/PHIMP/LSTM_sim_same_kge.csv', index=False)

In [ ]:
df_diff_lstm = pd.DataFrame(nse_scores_diff_lstm)
df_diff_lstm.to_csv('/content/drive/MyDrive/PHIMP/LSTM_sim_diff_nse.csv', index=False)

In [ ]:
df_diff_lstm = pd.DataFrame(kge_scores_diff_lstm)
df_diff_lstm.to_csv('/content/drive/MyDrive/PHIMP/LSTM_sim_diff_kge.csv', index=False)

In [ ]:
import tf_keras

In [ ]:
jj=2
ii=399
inputs = tf.keras.layers.Input(shape=(365, 9))
    #x=tf.keras.layers.Normalization(axis=-1)(inputs)
x = tf.keras.layers.LSTM(256, return_sequences=False)(inputs)
x=tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1,activation='linear')(x)
    #outputs=tf.keras.layers.Reshape((3,1),name='output')(outputs)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), loss="mse")
model.load_weights("/content/drive/MyDrive/PHIMP/LSTM_All_50_Weights_RandomNumbers/Final_sim_%d.h5"%(jj))
nn_without_head = tf.keras.models.Model(inputs=model.inputs, outputs=x)
#df_and_nn_model = tfdf.keras.RandomForestModel(preprocessing=nn_without_head,task = tfdf.keras.Task.REGRESSION)
df_and_nn_model=tf_keras.models.load_model('/content/drive/MyDrive/PHIMP/Tree models_Sim50/Tree_sim_%d_%d'%(jj,ii))

##Lead time 1

In [ ]:
df_tr = df_train.iloc[:,1:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)



for jj in range(9,10):
  print(jj)
  list_=list(pd.read_csv("/content/drive/MyDrive/PHIMP/LSTM_All_50_Weights_RandomNumbers/random_numbers_sim_%d.csv"%(jj)).iloc[:,1])


  nse_scores_same_lstm=[]
  kge_scores_same_lstm=[]

  nse_scores_diff_lstm=[]
  kge_scores_diff_lstm=[]
  inputs = tf.keras.layers.Input(shape=(365, 9))
    #x=tf.keras.layers.Normalization(axis=-1)(inputs)
  x = tf.keras.layers.LSTM(256, return_sequences=False)(inputs)
  x=tf.keras.layers.Dropout(0.2)(x)
  outputs = tf.keras.layers.Dense(1,activation='linear')(x)
    #outputs=tf.keras.layers.Reshape((3,1),name='output')(outputs)
  model = tf.keras.Model(inputs, outputs)
  model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), loss="mse")
  model.load_weights("/content/drive/MyDrive/PHIMP/LSTM_All_50_Weights_RandomNumbers/Final_sim_%d.h5"%(jj))
  nn_without_head = tf.keras.models.Model(inputs=model.inputs, outputs=x)
  #df_and_nn_model = tfdf.keras.RandomForestModel(preprocessing=nn_without_head,task = tfdf.keras.Task.REGRESSION)

  for ii in range(0,420):








      temp_x=np.asarray(df_tr[df_train['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
          'average_q', 'average_tmax', 'average_tmin']])
      temp_y=np.asarray(df_tr[df_train['basin_id']==ii]['q']).reshape((-1,1))
      xx,yy=split_sequence_multi_train(temp_x,temp_y,365,0,mode='seq')

      if (jj==9) and (ii<244):
        df_and_nn_model=tf_keras.models.load_model('/content/drive/MyDrive/PHIMP/Tree models_Sim50/Tree_sim_%d_%d'%(jj,ii))
      else:
        df_and_nn_model = tfdf.keras.RandomForestModel(preprocessing=nn_without_head,task = tfdf.keras.Task.REGRESSION,num_trees=100)
        df_and_nn_model.fit(x=xx,y=yy[:,0].reshape((-1,1)))
        df_and_nn_model.save('/content/drive/MyDrive/PHIMP/Tree models_Sim50/Tree_sim_%d_%d'%(jj,ii))




      temp_xx=np.asarray(df_test_tr[df_test_tr['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
          'average_q', 'average_tmax', 'average_tmin']])
      temp_yy=np.asarray(df_test_tr[df_test_tr['basin_id']==ii]['q']).reshape((-1,1))
      xx_,yy_=split_sequence_multi_train(temp_xx,temp_yy,365,0,mode='seq')




      y_out_tr=yy_[:,0].reshape((-1,1))
      y_pred_tr=df_and_nn_model.predict(xx_)
      y_m_out_tr=y_pred_tr.reshape((-1,1))

      y_out_tr=y_out_tr*std_q+mean_q
      y_m_out_tr=y_m_out_tr*std_q+mean_q

      if ii in list_:
        nse_scores_same_lstm.append(nash_sutcliffe_error(y_out_tr,y_m_out_tr[:,0].reshape((-1,1))))
        kge_scores_same_lstm.append(KGE(y_m_out_tr[:,0].reshape((-1,1)),y_out_tr))
      else:
        nse_scores_diff_lstm.append(nash_sutcliffe_error(y_out_tr,y_m_out_tr[:,0].reshape((-1,1))))
        kge_scores_diff_lstm.append(KGE(y_m_out_tr[:,0].reshape((-1,1)),y_out_tr))

  df_same_lstm = pd.DataFrame(nse_scores_same_lstm)
  df_same_lstm.to_csv('/content/drive/MyDrive/PHIMP/Tree_sim_same_nse_%d.csv'%(jj), index=False)

  df_same_lstm = pd.DataFrame(kge_scores_same_lstm)
  df_same_lstm.to_csv('/content/drive/MyDrive/PHIMP/Tree_sim_same_kge_%d.csv'%(jj), index=False)

  df_diff_lstm = pd.DataFrame(nse_scores_diff_lstm)
  df_diff_lstm.to_csv('/content/drive/MyDrive/PHIMP/Tree_sim_diff_nse_%d.csv'%(jj), index=False)

  df_diff_lstm = pd.DataFrame(kge_scores_diff_lstm)
  df_diff_lstm.to_csv('/content/drive/MyDrive/PHIMP/Tree_sim_diff_kge_%d.csv'%(jj), index=False)









9
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmpnw11km47 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:09.772077. Found 8208 examples.
Training model...
Model trained in 0:00:10.485615
Compiling model...
Model compiled.
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmpw8niomtb as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.969731. Found 8208 examples.
Training model...
Model trained in 0:00:10.311576
Compiling model...
Model compiled.
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmpzjgifwn8 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.803093. Found 8208 examples.
Training model...
Model trained in 0:00:10.406843
Compiling model...
Model compiled.
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmp905swezp as temporary training directory
Reading training dataset...
Training dataset 

Training dataset read in 0:00:04.882880. Found 8208 examples.
Training model...
Model trained in 0:00:10.369996
Compiling model...
Model compiled.


36/36 [==============================] - 1s 13ms/step
Use /tmp/tmp62wcdqse as temporary training directory
Reading training dataset...


Training dataset read in 0:00:04.903411. Found 8208 examples.
Training model...
Model trained in 0:00:10.209003
Compiling model...
Model compiled.


36/36 [==============================] - 1s 14ms/step
Use /tmp/tmpkta6udg6 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.909561. Found 8208 examples.
Training model...
Model trained in 0:00:10.388377
Compiling model...
Model compiled.
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmpzvovuu_2 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.890331. Found 8208 examples.
Training model...
Model trained in 0:00:10.301961
Compiling model...
Model compiled.
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmporwk7abw as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.843366. Found 8208 examples.
Training model...
Model trained in 0:00:10.561430
Compiling model...
Model compiled.
36/36 [==============================] - 1s 13ms/step
Use /tmp/tmpm1zvh1mv as temporary training directory
Reading training dataset...
Training dataset re

In [ ]:
ii

143

In [ ]:
nse_scores=np.zeros((len(list_),1))
for ii in list_:

  temp_x=np.asarray(df_tr[df_train['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
  temp_y=np.asarray(df_tr[df_train['basin_id']==ii]['q']).reshape((-1,1))
  xx,yy=split_sequence_multi_train(temp_x,temp_y,365,0,mode='seq')


  y_out=yy[:,0].reshape((-1,1))
  y_pred=model.predict(xx)
  y_m_out=y_pred.reshape((-1,1))

  y_out=i_normalize(y_out,mean_q,std_q)[:,0].reshape((-1,1))
  y_m_out=i_normalize(y_m_out,mean_q,std_q)

  df_y_out=pd.DataFrame(y_out)
  df_y_m_out=pd.DataFrame(y_m_out)
  df_y_out.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/Obs3_%dtrain.csv'%(ii), index=False)
  df_y_m_out.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/Sim3_%dtrain.csv'%(ii), index=False)


  nse_scores[list_.index(ii),0]=nash_sutcliffe_error(y_out,y_m_out[:,0].reshape((-1,1)))

nse_scores_df = pd.DataFrame(nse_scores)
nse_scores_df.to_csv('/content/drive/MyDrive/PHIMP/LSTM50_sim3_nse_train.csv', index=False)

257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
257/257 ━━━━━━━━

In [ ]:
#train
np.median(nse_scores,axis=0)

array([0.70188301])

In [ ]:
np.median(nse_scores_tr,axis=0)

array([0.68602474])

In [ ]:
np.median(kge_scores_tr)

0.7028876679479361

In [ ]:
np.mean(nse_scores_tr,axis=0)

array([0.58822238])

In [ ]:
list_all=[ii for ii in range(0,421)]

In [ ]:
# prompt: remove similar elements between random numbers and list_all, and save it in a new list
list_diff = list(set(list_all) - set(list_))

In [ ]:
LT=0
nse_scores=np.zeros((len(list_diff),1))
kge_scores_tr=np.zeros((len(list_diff),1))
for ii in list_diff:
  temp_x=np.asarray(df_test_tr[df_test_tr['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
  temp_y=np.asarray(df_test_tr[df_test_tr['basin_id']==ii]['q']).reshape((-1,1))
  xx,yy=split_sequence_multi_train(temp_x,temp_y,365,0,mode='seq')


  y_out=yy[:,0].reshape((-1,1))
  y_pred=model.predict(xx)
  y_m_out=y_pred.reshape((-1,1))

  y_out=i_normalize(y_out,mean_q,std_q)[:,0].reshape((-1,1))
  y_m_out=i_normalize(y_m_out,mean_q,std_q)

  df_y_out=pd.DataFrame(y_out)
  df_y_m_out=pd.DataFrame(y_m_out)
  df_y_out.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/LSTMObs3All%d.csv'%(ii), index=False)
  df_y_m_out.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/LSTMSim3All%d.csv'%(ii), index=False)


  nse_scores[list_diff.index(ii),0]=nash_sutcliffe_error(y_out,y_m_out[:,0].reshape((-1,1)))
  kge_scores_tr[list_diff.index(ii),0]=KGE(y_m_out[:,0].reshape((-1,1)),y_out)

nse_scores_df = pd.DataFrame(nse_scores)
nse_scores_df.to_csv('/content/drive/MyDrive/PHIMP/LSTMall_sim3_nse.csv', index=False)
kge_scores_tr_df = pd.DataFrame(kge_scores_tr,columns=['KGE'])
kge_scores_tr_df.to_csv('/content/drive/MyDrive/PHIMP/LSTMall_sim3_kge.csv', index=False)

np.median(nse_scores,axis=0)

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━

array([0.55109909])

In [ ]:
np.median(kge_scores_tr,axis=0)

array([0.58403617])

In [ ]:
np.mean(nse_scores,axis=0)

array([-34.8720524])

In [ ]:
df_tr = df_train.iloc[:,1:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
nse_scores_tr=np.zeros((len(list_diff),1))
kge_scores_tr=np.zeros((len(list_diff),1))
step=1
for ii in list_diff:






  temp_x=np.asarray(df_tr[df_train['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
  temp_y=np.asarray(df_tr[df_train['basin_id']==ii]['q']).reshape((-1,1))
  xx,yy=split_sequence_multi_train(temp_x,temp_y,365,0,mode='seq')

  df_and_nn_model = tfdf.keras.RandomForestModel(preprocessing=nn_without_head,task = tfdf.keras.Task.REGRESSION,num_trees=100)
  df_and_nn_model.fit(x=xx,y=yy[:,0].reshape((-1,1)))

  temp_xx=np.asarray(df_test_tr[df_test_tr['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
  temp_yy=np.asarray(df_test_tr[df_test_tr['basin_id']==ii]['q']).reshape((-1,1))
  xx_,yy_=split_sequence_multi_train(temp_xx,temp_yy,365,0,mode='seq')




  y_out_tr=yy_[:,0].reshape((-1,1))
  y_pred_tr=df_and_nn_model.predict(xx_)
  y_m_out_tr=y_pred_tr.reshape((-1,1))

  y_out_tr=y_out_tr*std_q+mean_q
  y_m_out_tr=y_m_out_tr*std_q+mean_q

  df_y_out_tr=pd.DataFrame(y_out_tr)
  df_y_m_out_tr=pd.DataFrame(y_m_out_tr)
  df_y_out_tr.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/ObsTrAll%d_sim3.csv'%(ii), index=False)
  df_y_m_out_tr.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/SimAll%d_sim3.csv'%(ii), index=False)



  nse_scores_tr[list_diff.index(ii)]=nash_sutcliffe_error(y_out_tr,y_m_out_tr)
  kge_scores_tr[list_diff.index(ii)]=KGE(y_m_out_tr,y_out_tr)
  #print(nse_scores_tr[list_.index(ii)])


nse_scores_tr_df = pd.DataFrame(nse_scores_tr,columns=['NSE'])
nse_scores_tr_df.to_csv('/content/drive/MyDrive/PHIMP/tr_nn_all_sim3_nse.csv', index=False)

kge_scores_tr_df = pd.DataFrame(kge_scores_tr,columns=['KGE'])
kge_scores_tr_df.to_csv('/content/drive/MyDrive/PHIMP/tr_nn_all_sim3_kge.csv', index=False)

Use /tmp/tmprn204v3g as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.430113. Found 8208 examples.
Training model...
Model trained in 0:00:07.145151
Compiling model...
Model compiled.
36/36 [==============================] - 1s 9ms/step
Use /tmp/tmpw2khek_y as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.304589. Found 8208 examples.
Training model...
Model trained in 0:00:07.165957
Compiling model...
Model compiled.
36/36 [==============================] - 1s 9ms/step
Use /tmp/tmpyxz3wapm as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.262788. Found 8208 examples.
Training model...
Model trained in 0:00:07.316195
Compiling model...
Model compiled.
36/36 [==============================] - 1s 9ms/step
Use /tmp/tmpjwtzpdw_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.263467. Found 8208 examples.
Training model

In [ ]:
np.median(nse_scores_tr,axis=0)

array([0.7281261])

In [ ]:
np.median(kge_scores_tr,axis=0)

array([0.73302868])

In [ ]:
df_tr = df_train.iloc[:,1:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
nse_scores_tr=np.zeros((len(list_diff),1))
kge_scores_tr=np.zeros((len(list_diff),1))

def modeli():
  inputs = tf.keras.layers.Input(shape=(365, 9))
  #x=tf.keras.layers.Normalization(axis=-1)(inputs)
  x = tf.keras.layers.LSTM(256, return_sequences=False)(inputs)
  x=tf.keras.layers.Dropout(0.2)(x)
  outputs = tf.keras.layers.Dense(1,activation='linear')(x)
  #outputs=tf.keras.layers.Reshape((3,1),name='output')(outputs)
  model_ = tf.keras.Model(inputs, outputs)
  return model_



for ii in list_diff:






  temp_x=np.asarray(df_tr[df_train['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
  temp_y=np.asarray(df_tr[df_train['basin_id']==ii]['q']).reshape((-1,1))
  xx,yy=split_sequence_multi_train(temp_x,temp_y,365,0,mode='seq')


  model_=modeli()
  model_.load_weights("/content/drive/MyDrive/PHIMP/Checkpoint/Final50_sim.h5")
  model_.compile(optimizer=tf.optimizers.Adam(learning_rate=5e-6), loss="mse")


  history = model_.fit(x=xx,y=yy, epochs=5,batch_size=32)
  model_.save_weights("/content/drive/MyDrive/PHIMP/Checkpoint/simlstmfine_%d.weights.h5"%(ii))

  temp_xx=np.asarray(df_test_tr[df_test_tr['basin_id']==ii].loc[:, ['pr', 'srad', 'tmax', 'tmin', 'vp', 'average_pr',
       'average_q', 'average_tmax', 'average_tmin']])
  temp_yy=np.asarray(df_test_tr[df_test_tr['basin_id']==ii]['q']).reshape((-1,1))
  xx_,yy_=split_sequence_multi_train(temp_xx,temp_yy,365,0,mode='seq')




  y_out_tr=yy_[:,0].reshape((-1,1))
  y_pred_tr=model_.predict(xx_)
  y_m_out_tr=y_pred_tr.reshape((-1,1))

  y_out_tr=y_out_tr*std_q+mean_q
  y_m_out_tr=y_m_out_tr*std_q+mean_q

  df_y_out_tr=pd.DataFrame(y_out_tr)
  df_y_m_out_tr=pd.DataFrame(y_m_out_tr)
  df_y_out_tr.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/ObsLSTMFine%d_sim.csv'%(ii), index=False)
  df_y_m_out_tr.to_csv('/content/drive/MyDrive/PHIMP/Forecasts_CAMELS/LSTMFine%d_sim.csv'%(ii), index=False)



  nse_scores_tr[list_diff.index(ii)]=nash_sutcliffe_error(y_out_tr,y_m_out_tr)
  kge_scores_tr[list_diff.index(ii)]=KGE(y_m_out_tr,y_out_tr)
  #print(nse_scores_tr[list_.index(ii)])


nse_scores_tr_df = pd.DataFrame(nse_scores_tr,columns=['NSE'])
nse_scores_tr_df.to_csv('/content/drive/MyDrive/PHIMP/LSTMFine.csv', index=False)

kge_scores_tr_df = pd.DataFrame(kge_scores_tr,columns=['KGE'])
kge_scores_tr_df.to_csv('/content/drive/MyDrive/PHIMP/LSTMFine_kge.csv', index=False)

Epoch 1/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 12s 43ms/step - loss: 0.1820
Epoch 2/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.0993
Epoch 3/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - loss: 0.0886
Epoch 4/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.0880
Epoch 5/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.0718
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step
Epoch 1/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 0.1443
Epoch 2/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.1756
Epoch 3/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.0997
Epoch 4/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - loss: 0.1076
Epoch 5/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - loss: 0.0839
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
Epoch 1/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.1461
Epoch 2/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.1313
Epoch 3/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.1353
Epoch 4/5
257/257 ━━━━━━━━━━━━━━━━━━━━ 9s 35m

In [ ]:
np.median(nse_scores_tr,axis=0)

array([0.71539226])

In [ ]:
np.median(kge_scores_tr)

0.7379746638970837

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 365, 9)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 256)                 │         272,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 272,641 (1.04 MB)

 Trainable params: 272,641 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_.summary()

AttributeError: 'NoneType' object has no attribute 'summary'